<a href="https://colab.research.google.com/github/Tim3s/PL-teamproject-dataprocessing/blob/main/dataprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
setting neccessary paths to check file, and list of values for each column of user data
must execute every time you start this program, or modify any variables
"""

# path to user_profile.xlsx
user_profile_path = "drive/MyDrive/PL_team_project/user_profile.xlsx"

# path to user data files
user_data_path_before_id = 'drive/MyDrive/PL_team_project/hs_'
user_data_path_after_id = '_m08_0903_1355.csv'

# encoding option for user data file
user_data_encoder = 'cp949'

# all value for 'State' column of user data files
state = ['TV 보기', '간식 먹기', '간편식 준비하기', '귀가하기', '기상 약 복용', '기상하기', '꿀잠소리', '날씨 확인하기', '낮잠 기상하기', '낮잠자기', '냉장고 정리하기', '노래자랑', '뉴스 시청', '늦은 기상하기', '도전 실버벨', '듣는대화', '마음그림터', '마음세탁소', '마음스트레칭', '명언산책', '무비순이', '밥솥 열기', '밥솥 정리하기', '새벽 TV 시청', '새벽 간식', '새벽 화장실 이용', '석식', '설거지 하기', '수면', '순이 특별대화', '순이극장', '순이대화', '순이인생', '순이책방', '순이체조', '시낭독', '시시콜콜', '식사', '식사 중 TV 시청', '식후 약 복용', '실내운동하기', '실외운동하기', '야식 먹기', '영어교실', '외출하기', '요가명상', '운동 후 귀가', '음식 꺼내기', '이른 기상하기', '일반 약 복용', '일어교실', '전자렌지 청소하기', '조식', '주방 정리하기', '중식', '집 환기하기', '취미활동', '화장실 이용']

# activeness rate of each state except programs(1~4).
# you don't have to classify all, but it will be considered as data insufficience if none of them is in file.
activeness4 = ['외출하기', '운동 후 귀가', '이른 기상하기']
activeness3 = ['귀가하기', '설거지 하기', '전자렌지 청소하기', '주방 정리하기', '집 환기하기', '취미활동']
activeness2 = ['간식 먹기', '간편식 준비하기', '새벽 TV 시청']
activeness1 = ['TV 보기', '낮잠자기', '뉴스 시청', '늦은 기상하기']

# all program list and activeness rate of each program(1~4)
# must classify all programs, and at least one program must be classified as each activeness rate
program = ['꿀잠소리', '노래자랑', '도전 실버벨', '듣는대화', '마음그림터', '마음세탁소', '마음스트레칭', '명언산책', '무비순이', '순이 특별대화', '순이극장', '순이대화', '순이인생', '순이책방', '순이체조', '시낭독', '시시콜콜', '영어교실', '요가명상', '일어교실']
program_activeness4 = ['노래자랑', '순이체조']
program_activeness3 = ['도전 실버벨', '마음세탁소', '마음스트레칭', '영어교실', '요가명상', '일어교실']
program_activeness2 = ['듣는대화', '마음그림터', '순이 특별대화', '순이대화', '순이인생', '순이책방', '시시콜콜']
program_activeness1 = ['꿀잠소리', '명언산책', '무비순이', '순이극장', '시낭독']

# classifying all state into 4 to use for classifying users into 4 groups
in_act = ['TV 보기']
hobby = ['취미활동']
basic = ['음식 꺼내기']
out_act = ['외출하기']

# path to save each files
overall_validness_path = 'drive/MyDrive/PL_team_project/processed_data/overall_validness.csv'
active_frequency_path = 'drive/MyDrive/PL_team_project/processed_data/active_frequency.csv'
cluster_path = 'drive/MyDrive/PL_team_project/processed_data/cluster.csv'
activeness_path = 'drive/MyDrive/PL_team_project/processed_data/activeness.csv'
regularity_path = 'drive/MyDrive/PL_team_project/processed_data/regularity.csv'
program_path = 'drive/MyDrive/PL_team_project/processed_data/program.csv'
response_path = 'drive/MyDrive/PL_team_project/processed_data/response.csv'
friendship_path = 'drive/MyDrive/PL_team_project/processed_data/friendship.csv'

In [2]:
"""
basic library import and process initialized variables
must execute every time you start this program, or modify any variables
"""

import numpy as np
import pandas as pd
import datetime as dt
from sklearn.cluster import KMeans
import random
from bisect import bisect_right


user_profile_csv = pd.read_excel(user_profile_path)
user_profile = user_profile_csv.to_numpy()
state_np = np.array(state)
program_np = np.array(program)
program_activeness = dict()
for program_name in program:
    if program_name in program_activeness4:
        program_activeness[program_name] = 4
    elif program_name in program_activeness3:
        program_activeness[program_name] = 3
    elif program_name in program_activeness1:
        program_activeness[program_name] = 1
    else:
        program_activeness[program_name] = 2

valid = []
valid_age = []
invalid = []
small_data = []
active_input = []

for user_profile_data in user_profile:
    id = user_profile_data[0]
    try:
        user_data_csv = pd.read_csv(user_data_path_before_id + str(id) + user_data_path_after_id, encoding=user_data_encoder)
    except:
        invalid.append(id)
        print('file not found:', id)
        continue

    user_data = user_data_csv.to_numpy()
    if len(user_data) <= 10:
        small_data.append(id)
        print('not enough data:', id)
        continue
    
    time = pd.to_datetime(user_data_csv['Time'])
    time = time[len(time)-1] - time[0]
    if time < dt.timedelta(seconds=86400):
        small_data.append(id)
        print('not enough data:', id)
        continue

    state_data = user_data[:,4]
    activeness = [0, 0, 0, 0]
    for action in activeness4:
        activeness[0] += np.sum(state_data==action)
    for action in activeness3:
        activeness[1] += np.sum(state_data==action)
    for action in activeness2:
        activeness[2] += np.sum(state_data==action)
    for action in activeness1:
        activeness[3] += np.sum(state_data==action)
    for pro in program_activeness4:
        activeness[0] += np.sum(state_data==pro)
    for pro in program_activeness3:
        activeness[1] += np.sum(state_data==pro)
    for pro in program_activeness2:
        activeness[2] += np.sum(state_data==pro)
    for pro in program_activeness1:
        activeness[3] += np.sum(state_data==pro)
    if not sum(activeness):
        small_data.append(id)
        print('not enough data:', id)
        continue
    active_input.append((sum([activeness[i] * (4 - i) for i in range(4)]) / sum(activeness)))
    valid.append(id)
    valid_age.append(user_profile_data[2])

not enough data: 273
not enough data: 342
not enough data: 504
not enough data: 509
not enough data: 530
not enough data: 532
not enough data: 537
not enough data: 538
not enough data: 540
not enough data: 552
not enough data: 574
not enough data: 633
not enough data: 635
not enough data: 636
not enough data: 653
not enough data: 664
not enough data: 667
not enough data: 732
not enough data: 804
not enough data: 808
not enough data: 1011
not enough data: 1034
not enough data: 1039
not enough data: 1045
not enough data: 1056
not enough data: 30043
not enough data: 30044
not enough data: 30047
not enough data: 30049
not enough data: 30052
not enough data: 30059
not enough data: 30061
not enough data: 30062
file not found: 30064
file not found: 30066
file not found: 30067
file not found: 30069
file not found: 30070
file not found: 30071
file not found: 30072
file not found: 30073
file not found: 30074
file not found: 30075
file not found: 30076
file not found: 30077
file not found: 30078


In [3]:
"""
make file for overall validness.
must make all other files, because this file will affect the structure of other files.
"""

f = open(overall_validness_path, 'w')
f.write('id,validness')
for id in user_profile_csv['id']:
    if id in invalid:
        f.write('\n' + str(id) + ',invalid')
    elif id in small_data:
        f.write('\n' + str(id) + ',too small data')
    else:
        f.write('\n' + str(id) + ',valid')
f.close()

In [4]:
"""
make file for average percentage of each action people do.
"""

state_input = np.empty(shape=(0, len(state)))
for id in valid:
    user_data_csv = pd.read_csv(user_data_path_before_id + str(id) + user_data_path_after_id, encoding=user_data_encoder)
    result = np.sum([state_np==i for i in user_data_csv['State']], axis=0)
    result = result / np.sum(result)
    state_input = np.append(state_input, np.array([result]), axis=0)

f = open(active_frequency_path, 'w')
f.write(','.join(state))
f.write('\n')
f.write(','.join(map(str, np.mean(state_input, axis=0))))
f.close()

In [5]:
"""
make file for classifying people into 4 clusters.
"""

state_input = np.empty(shape=(0, len(state)))
for id in valid:
    user_data_csv = pd.read_csv(user_data_path_before_id + str(id) + user_data_path_after_id, encoding=user_data_encoder)
    result = np.sum([state_np==i for i in user_data_csv['State']], axis=0)
    result = result / np.sum(result)
    state_input = np.append(state_input, np.array([result]), axis=0)

km = KMeans(n_clusters=4)
km.fit(state_input)

valid_np = np.array(valid)
classify = {'실내활동': [], '야외활동': [], '일상활동': [], '취미활동': []}
for i in range(km.n_clusters):
    if state[np.argmax(np.mean(state_input[km.labels_==i], axis=0))] in in_act:
        classify['실내활동'] = np.append(classify['실내활동'], valid_np[km.labels_==i])
    elif state[np.argmax(np.mean(state_input[km.labels_==i], axis=0))] in out_act:
        classify['야외활동'] = np.append(classify['야외활동'], valid_np[km.labels_==i])
    elif state[np.argmax(np.mean(state_input[km.labels_==i], axis=0))] in hobby:
        classify['취미활동'] = np.append(classify['취미활동'], valid_np[km.labels_==i])
    elif state[np.argmax(np.mean(state_input[km.labels_==i], axis=0))] in basic:
        classify['일상활동'] = np.append(classify['일상활동'], valid_np[km.labels_==i])

f = open(cluster_path, 'w')
f.write('id,activity,' + ','.join(state))
f.write('\n0,null,' + ','.join(map(str, np.mean(state_input, axis=0))))
for i in range(len(valid)):
    id = valid[i]
    written = False
    for key in classify:
        if id in classify[key]:
            f.write('\n' + str(id) + ',' + key)
            written = True
            break
    if not written:
        f.write('\n' + str(id) + ',classify failed')
    f.write(',' + ','.join(map(str, state_input[i])))
f.close()

In [6]:
"""
make file for checking score and rank of each users' activeness
"""

active_input = active_input / np.max(active_input) * 100

fifty_active = []
sixty_active = []
seventy_active = []
eighty_active = []
for i in range(len(valid_age)):
    if valid_age[i] < 60:
        fifty_active.append(active_input[i])
    elif valid_age[i] < 70:
        sixty_active.append(active_input[i])
    elif valid_age[i] < 80:
        seventy_active.append(active_input[i])
    else:
        eighty_active.append(active_input[i])
fifty_active.sort()
sixty_active.sort()
seventy_active.sort()
eighty_active.sort()

f = open(activeness_path, 'w')
f.write('id,age,score,rank,total')
for i in range(len(valid)):
    if valid_age[i] < 60:
        f.write('\n' + str(valid[i]) + ',50,' + str(active_input[i]) + ',' + str(len(fifty_active) - bisect_right(fifty_active, active_input[i]) + 1) + ',' + str(len(fifty_active)))
    elif valid_age[i] < 70:
        f.write('\n' + str(valid[i]) + ',60,' + str(active_input[i]) + ',' + str(len(sixty_active) - bisect_right(sixty_active, active_input[i]) + 1) + ',' + str(len(sixty_active)))
    elif valid_age[i] < 80:
        f.write('\n' + str(valid[i]) + ',70,' + str(active_input[i]) + ',' + str(len(seventy_active) - bisect_right(seventy_active, active_input[i]) + 1) + ',' + str(len(seventy_active)))
    else:
        f.write('\n' + str(valid[i]) + ',80,' + str(active_input[i]) + ',' + str(len(eighty_active) - bisect_right(eighty_active, active_input[i]) + 1) + ',' + str(len(eighty_active)))
f.close()

In [7]:
"""
make file for checking regularity of each user
"""

pattern_valid = []
no_pattern = []
pattern_input = []
wake_valid = []
wake_input = []
break_valid = []
break_input = []
lunch_valid = []
lunch_input = []
dinner_valid = []
dinner_input = []
for id in valid:
    user_data_csv = pd.read_csv(user_data_path_before_id + str(id) + user_data_path_after_id, encoding=user_data_encoder)
    user_data = user_data_csv.to_numpy()
    state_data = user_data[:,4]
    time_data_csv = user_data_csv['Time']
    if ('기상하기' not in state_data) and ('늦은 기상하기' not in state_data) and ('이른 기상하기' not in state_data) and ('조식' not in state_data) and ('중식' not in state_data) and ('석식' not in state_data):
        no_pattern.append(id)
    else:
        pattern_valid.append(id)
        if ('기상하기' in state_data) or ('늦은 기상하기' in state_data) or ('이른 기상하기' in state_data):
            wake_valid.append(id)
            current = []
            for i in range(len(state_data)):
                cur_data = state_data[i]
                if cur_data == '기상하기' or cur_data == '늦은 기상하기' or cur_data == '이른 기상하기':
                    current.append(pd.to_datetime(time_data_csv[i]).to_pydatetime())
                    current[-1] = dt.timedelta(hours=(current[-1].hour if current[-1].hour < 12 else (-24 + current[-1].hour)), minutes=current[-1].minute).seconds
            wake_input.append(np.std(current))
        if '조식' in state_data:
            break_valid.append(id)
            current = []
            for i in range(len(state_data)):
                cur_data = state_data[i]
                if cur_data == '조식':
                    current.append(pd.to_datetime(time_data_csv[i]).to_pydatetime())
                    current[-1] = dt.timedelta(hours=current[-1].hour, minutes=current[-1].minute).seconds
            break_input.append(np.std(current))
        if '중식' in state_data:
            lunch_valid.append(id)
            current = []
            for i in range(len(state_data)):
                cur_data = state_data[i]
                if cur_data == '중식':
                    current.append(pd.to_datetime(time_data_csv[i]).to_pydatetime())
                    current[-1] = dt.timedelta(hours=current[-1].hour, minutes=current[-1].minute).seconds
            lunch_input.append(np.std(current))
        if '석식' in state_data:
            dinner_valid.append(id)
            current = []
            for i in range(len(state_data)):
                cur_data = state_data[i]
                if cur_data == '석식':
                    current.append(pd.to_datetime(time_data_csv[i]).to_pydatetime())
                    current[-1] = dt.timedelta(hours=current[-1].hour, minutes=current[-1].minute).seconds
            dinner_input.append(np.std(current))

wake_input = np.array(wake_input) / max(wake_input)
break_input = np.array(break_input) / max(break_input)
lunch_input = np.array(lunch_input) / max(lunch_input)
dinner_input = np.array(dinner_input) / max(dinner_input)
wakeptr = breakptr = lunchptr = dinnerptr = 0
for id in pattern_valid:
    current = []
    if wakeptr < len(wake_valid) and wake_valid[wakeptr] == id:
        current.append(1 - wake_input[wakeptr])
        wakeptr += 1
    if breakptr < len(break_valid) and break_valid[breakptr] == id:
        current.append(1 - break_input[breakptr])
        breakptr += 1
    if lunchptr < len(lunch_valid) and lunch_valid[lunchptr] == id:
        current.append(1 - lunch_input[lunchptr])
        lunchptr += 1
    if dinnerptr < len(dinner_valid) and dinner_valid[dinnerptr] == id:
        current.append(1 - dinner_input[dinnerptr])
        dinnerptr += 1
    pattern_input.append(np.mean(current) * 100)

f = open(regularity_path, 'w')
f.write('id,validness,score')
for id in valid:
    if id in no_pattern:
        f.write('\n' + str(id) + ',0,0')
        continue
    f.write('\n' + str(id) + ',1,' + str(pattern_input[pattern_valid.index(id)]))
f.close()

In [8]:
"""
make file for displaying each users' program participation and recommend a program
"""

program_valid = []
no_program = []
program_input = []
for id in valid:
    user_data_csv = pd.read_csv(user_data_path_before_id + str(id) + user_data_path_after_id, encoding=user_data_encoder)
    act_data = user_data_csv['Z'].to_numpy()
    state_data = user_data_csv['State'].to_numpy()
    if ('프로그램' not in act_data):
        no_program.append(id)
    else:
        program_valid.append(id)
        program_input.append([np.sum(state_data==i) for i in program])
        program_input[-1].append(sum(program_input[-1]))

f = open(program_path, 'w')
f.write('id,validness,' +  ','.join(program) + ',total,recommend')
program_valid_activeness = [active_input[i] for i in range(len(active_input)) if valid[i] in program_valid]
tmpmean = np.mean(program_valid_activeness)
tmpstd = np.std(program_valid_activeness)
for id in valid:
    f.write('\n' + str(id) + ',')
    if id in no_program:
        f.write('0,' * (len(program) + 2))
        curactive = active_input[valid.index(id)]
        if curactive > np.mean(active_input) + np.std(active_input):
            f.write(random.choice(program_activeness1))
        elif curactive > np.mean(active_input):
            f.write(random.choice(program_activeness2))
        elif curactive > np.mean(active_input) - np.std(active_input):
            f.write(random.choice(program_activeness3))
        else:
            f.write(random.choice(program_activeness4))
        continue
    idx = program_valid.index(id)
    f.write('1,' + ','.join(map(str, program_input[idx])))
    if program_valid_activeness[idx] > tmpmean + tmpstd:
        recommendlist = [program_input[idx][program.index(pro)] for pro in program_activeness1]
        recommendlist = np.array(program_activeness1)[np.array(recommendlist)==np.min(recommendlist)]
        f.write(',' + random.choice(recommendlist))
    elif program_valid_activeness[idx] > tmpmean:
        recommendlist = [program_input[idx][program.index(pro)] for pro in program_activeness2]
        recommendlist = np.array(program_activeness2)[np.array(recommendlist)==np.min(recommendlist)]
        f.write(',' + random.choice(recommendlist))
    elif program_valid_activeness[idx] > tmpmean - tmpstd:
        recommendlist = [program_input[idx][program.index(pro)] for pro in program_activeness3]
        recommendlist = np.array(program_activeness3)[np.array(recommendlist)==np.min(recommendlist)]
        f.write(',' + random.choice(recommendlist))
    else:
        recommendlist = [program_input[idx][program.index(pro)] for pro in program_activeness4]
        recommendlist = np.array(program_activeness4)[np.array(recommendlist)==np.min(recommendlist)]
        f.write(',' + random.choice(recommendlist))
f.close()

In [9]:
"""
make file for checking each users' response rate
"""

response_valid = []
response_invalid = []
num_of_conversation = []
for id in valid:
    user_data_csv = pd.read_csv(user_data_path_before_id + str(id) + user_data_path_after_id, encoding=user_data_encoder)
    result = [len(user_data_csv['STT_1'].dropna()), len(user_data_csv['STT_2'].dropna()), len(user_data_csv['STT_3'].dropna())]
    response = len([i for i in user_data_csv['Message_1'].dropna() if i != '프로그램 메시지'])
    if not response:
        response_invalid.append(id)
        result += [0] * 6
    else:
        response_valid.append(id)
        result.append(response)
        result.append(result[0] / response * 100 if response else 0)
        response = len([i for i in user_data_csv['Message_2'].dropna() if i != '프로그램 메시지'])
        result.append(response)
        result.append(result[1] / response * 100 if response else 0)
        response = len([i for i in user_data_csv['Message_3'].dropna() if i != '프로그램 메시지'])
        result.append(response)
        result.append(result[2] / response * 100 if response else 0)
    num_of_conversation.append(result)

f = open(response_path, 'w')
f.write('id,validness,response1,response2,response3,total1,percentage1,total2,percentage2,total3,percentage3')
for i in range(len(valid)):
    id = valid[i]
    f.write('\n' + str(id) + ',' + str(int(id not in response_invalid)) + ',' + ','.join(map(str, num_of_conversation[i])))
f.close()

In [18]:
"""
make file for checking friendship with 순이
must make program.csv and response.csv first, since this file will depend on the result of these files
"""

f = open(friendship_path, 'w')
f.write('id,score,rank')
score = []
for id in valid:
    cur = 0
    if id in program_valid:
        idx = program_valid.index(id)
        for i in range(len(program_input[idx]) - 1):
            cur += 30 * (1 - (0.664 ** program_input[idx][i]))
    if id in response_valid:
        idx = valid.index(id)
        cur += 400 * (1 - 0.993 ** (num_of_conversation[idx][0] + num_of_conversation[idx][1] + num_of_conversation[idx][2]))
    score.append(cur)
sorted_score = sorted(score)
for i in range(len(valid)):
    id = valid[i]
    f.write('\n' + str(id) + ',' + str(score[i]) + ',' + str(len(valid) - bisect_right(sorted_score, score[i]) + 1))
f.close()